In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString

import vdmlab as vdm

from load_data import get_pos, get_raw_pos, get_events
from maze_functions import spikes_by_position
from plotting_functions import plot_intersects, plot_zone

import sys
sys.path.append('E:\\code\\python-vdmlab\\projects\\emily_shortcut\\info')
import info.R063d2_info as r063d2

In [ ]:
# output_path = 'C:\\Users\\Emily\\Code\\emi_shortcut\\cache\\matlab\\spike_pos\\'
output_path = 'E:\\code\\emi_shortcut\\cache\\matlab\\spike_pos\\'
info = r063d2

In [ ]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)
raw = get_raw_pos(info.raw_pos_mat, info.pxl_to_cm)
events = get_events(info.event_mat)

In [ ]:
# Plot to check
plt.plot(pos.x, pos.y, 'b.', ms=1)
plt.show()

In [ ]:
# Plot to check
plt.plot(raw.x, raw.y, 'b.', ms=1)
plt.show()

In [ ]:
len(events['led2'])

In [ ]:
events.keys()

In [ ]:
for event in events['feederoff']:
    idx = vdm.find_nearest_idx(raw.time, event)
    plt.plot(raw.x[idx], raw.y[idx], 'r.', ms=2)
    plt.plot(raw.x[idx-1], raw.y[idx-2], 'b.', ms=2)
    plt.plot(raw.x[idx-1], raw.y[idx-3], 'c.', ms=2)
plt.show()

In [ ]:
for key in events:
    print()

In [ ]:
expand_by = 5
feeder1_center = Point(info.path_pts['feeder1'][0], info.path_pts['feeder1'][1])
feeder1 = feeder1_center.buffer(expand_by*2)
feeder2_center = Point(info.path_pts['feeder2'][0], info.path_pts['feeder2'][1])
feeder2 = feeder2_center.buffer(expand_by*2)

In [ ]:
plt.plot(raw.x, raw.y, 'g.', ms=1)
plot_zone(feeder1)
plot_zone(feeder2)
plt.show()

In [ ]:
def correct_position(position, events, feeder1, feeder2):
    """Removes positions above a certain speed threshold

    Parameters
    ----------
    position : vdmlab.Position
    events: dict
        With led1, feederoff, type, feeder1, pb2, ledoff, led2, pboff, label, pb1, feeder2 as keys.

    Returns
    -------
    filtered_position : vdmlab.Position

    """
    remove_idx = []
    for event in events['feederoff']:
        idx = vdm.find_nearest_idx(raw.time, event)
        for i in range(5):
            point = Point([pos.x[idx-i], pos.y[idx-i]])
            if feeder1.contains(point) or feeder2.contains(point):
                remove_idx.append(i)
    
    return position[~np.hstack(remove_idx)]

In [ ]:
test = correct_position(raw, events, feeder1, feeder2)

In [ ]:
plt.plot(test.x, test.y, 'g.', ms=1)
plt.show()